<a href="https://colab.research.google.com/github/alexv710/debias_cv_data/blob/main/DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import io

In [2]:
# temp_data = pd.io.stata.read_stata("data_candidates_sample_tab6.dta")
# temp_data.to_csv('data_candidates_sample_tab6.csv')

# temp_data = pd.io.stata.read_stata("data_candidates_sample_tab7_tabA9.dta")
# temp_data.to_csv('data_candidates_sample_tab7_tabA9.csv')

# temp_data = pd.io.stata.read_stata("data_recruiters.dta")
# temp_data.to_csv('data_recruiters.csv')

In [3]:
# read the mainsample.dta file for further processing 
data = pd.io.stata.read_stata("data_candidates_mainsample.dta")
data.to_csv('data_candidates_mainsample.csv')

In [4]:
# Import the mainsample via StataReader to use variable_labels (old version of the read_stata)
data_stata = pd.io.stata.StataReader("data_candidates_mainsample.dta")
data_labels = data_stata.variable_labels()

In [5]:
# Write the Labels into a csv
#import csv

# with open('dataLabels.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
#    w = csv.DictWriter(f, data_labels.keys())
#    w.writeheader()
#    w.writerow(data_labels)

In [6]:
# Remove all columns in the dataset for which no labels exist
# Those are mostly columns needed for p-tests and some matrix calculations in
# Stata. The outputted labels are those that are kept in the data

data = data.drop(columns=['CVAxORIGINE_IM_12', 'CVAxZUS_CUCS', 'CVAxZetI', 'C', 'CVA0', 'p_offre1', 'control_manquant', 'ZouI_pred', 'ZouI0', 'ALE_16'])

# Remove sampling weights (within and out of the experiment)

data = data.drop(columns='POIDS_SEL')

# Remove all the centered features

data = data.drop(columns=['FEMME_c', 'a30m_c', 'a3049_c', 'a50p_c', 'dip_aucun_c', 'dip_bepcap_c', 'dip_bac_c', 'dip_bac2p_c', 'recherche_LD_c', 'recherche_TLD_c'])

# Remove all features that are not directly derivable from the CVs

data = data.drop(columns=['ORIGINE_IM_1', 'ORIGINE_IM_2', 'ORIGINE_IM_12', 'ID_OFFRE', 'ID_CANDIDAT', ])

for key, value in data_labels.items():
  for col in data.columns:
    if key==col:
      if len(value)==0 or value.startswith('ID_OFFRE=='):
        data = data.drop(columns=col)
      else:
        # Print all the columns that are included in the dataset
        print(key + ':      ', value)

CVA:       Treatment: anonymous resume
REFUSAL:       Recruiter refused the experiment
ENTRETIEN:       Interviewed
RECRUTE:       Hired
PREN_MUSULMAN:       Muslim souding name
ZUS_CUCS:       Deprived neighborhood
ZouI:       Minority (immigrant or child of immigrant or residing in deprived neighborhood)
ZetI:       Residing in deprived neighborhood and with foreign bachkground (child of or immi
FEMME:       Female candidate
a50p:       Candidate over 50 years old
a3049:       Candidate between 30 and 49 years old
a26m:       Candidate below 26 years old
dip_aucun:       No diploma
dip_bepcap:       Professional degree
dip_bac:       High school diploma
dip_bac2p:       Upper education degree
dip_bac2:       L2 (diploma 2 years after high school)
dip_bac3p:       At least L3 (diploma 3 years after high school)
duree_expro_offre:       Work experience for the job advertised (in years)
recherche_LD:       Candidate has been looking for a job for at least one year
recherche_TLD:       C

In [7]:
X_ano = pd.DataFrame()
X_unano = pd.DataFrame()

for index, row in data.iterrows():
  if row['CVA'] == 1: 
    X_ano = X_ano.append(row)
  else:
    X_unano = X_unano.append(row)

# Assign the interviewed column to our target vector
y_ano = X_ano['ENTRETIEN']
y_unano = X_unano['ENTRETIEN']

# Drop the Interviewed/hired & anonymized columns (hired is much dependend on the number of jobs available
# and will currently not be considered as target label)
X_ano = data.drop(columns=['ENTRETIEN', 'RECRUTE', 'CVA'])
X_unano = X_unano.drop(columns=['ENTRETIEN', 'RECRUTE', 'CVA'])

In [8]:
X_unano.to_csv('X_unano.csv')

In [9]:
import os
import datetime
import numpy as np
import pandas as pd
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [10]:
# Train, Test split

X_train, X_test, y_train, y_test = train_test_split(X_unano, y_unano, test_size=0.33, random_state=42)

input_shape = X_train[:1].shape

In [22]:
X_train.columns

Index(['ALE_1', 'ALE_10', 'ALE_11', 'ALE_12', 'ALE_13', 'ALE_14', 'ALE_15',
       'ALE_2', 'ALE_3', 'ALE_4', 'ALE_5', 'ALE_6', 'ALE_7', 'ALE_8', 'ALE_9',
       'CDI', 'CNT_sup6m', 'CV_attrayant_', 'CV_experience_',
       'CV_experience_3p', 'CV_formation_', 'CV_hesit', 'CV_hesit_',
       'CV_inactivite', 'CV_note_2', 'CV_note_3', 'CV_note_4', 'CV_note_5',
       'CV_note_7p', 'CV_qualif', 'CV_qualif_inf', 'CV_qualif_sup', 'FEMME',
       'PREN_MUSULMAN', 'REFUSAL', 'SALREV_SMIC_2', 'ZUS_CUCS', 'ZetI', 'ZouI',
       'a26m', 'a3049', 'a50p', 'cadre', 'codage_cv', 'construction',
       'dip_aucun', 'dip_bac', 'dip_bac2', 'dip_bac2p', 'dip_bac3p',
       'dip_bepcap', 'duree_expro_offre', 'effent_200plus', 'eoq',
       'etudes_etranger', 'industrie', 'langue_anglais', 'langue_arabe',
       'langue_autres', 'permis', 'poste_unique', 'profint', 'recherche_LD',
       'recherche_TLD', 'service_m', 'service_nm', 'travail_etranger'],
      dtype='object')

In [30]:
# define the keras model

name="simpleNet"
model = keras.Sequential(
    [
        layers.Dense(32, input_dim=67, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(1, activation='sigmoid')  
    
    ],name=name
)
model.summary()

Model: "simpleNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 32)                2176      
_________________________________________________________________
dense_13 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 17        
Total params: 2,721
Trainable params: 2,721
Non-trainable params: 0
_________________________________________________________________


In [43]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'mse'])

In [44]:
model.fit(X_train, y_train, epochs=150, batch_size=10)

Epoch 1/150
58/58 [==============================] - 0s 569us/step - loss: nan - accuracy: 0.8858 - mse: nan
Epoch 2/150
58/58 [==============================] - 0s 586us/step - loss: nan - accuracy: 0.8858 - mse: nan
Epoch 3/150
58/58 [==============================] - 0s 586us/step - loss: nan - accuracy: 0.8858 - mse: nan
Epoch 4/150
58/58 [==============================] - 0s 569us/step - loss: nan - accuracy: 0.8858 - mse: nan
Epoch 5/150
58/58 [==============================] - 0s 604us/step - loss: nan - accuracy: 0.8858 - mse: nan
Epoch 6/150
58/58 [==============================] - 0s 575us/step - loss: nan - accuracy: 0.8858 - mse: nan
Epoch 7/150
58/58 [==============================] - 0s 569us/step - loss: nan - accuracy: 0.8858 - mse: nan
Epoch 8/150
58/58 [==============================] - 0s 568us/step - loss: nan - accuracy: 0.8858 - mse: nan
Epoch 9/150
58/58 [==============================] - 0s 601us/step - loss: nan - accuracy: 0.8858 - mse: nan
Epoch 10/150
58/58 

58/58 [==============================] - 0s 619us/step - loss: nan - accuracy: 0.8858 - mse: nan
Epoch 76/150
58/58 [==============================] - 0s 587us/step - loss: nan - accuracy: 0.8858 - mse: nan
Epoch 77/150
58/58 [==============================] - 0s 569us/step - loss: nan - accuracy: 0.8858 - mse: nan
Epoch 78/150
58/58 [==============================] - 0s 541us/step - loss: nan - accuracy: 0.8858 - mse: nan
Epoch 79/150
58/58 [==============================] - 0s 577us/step - loss: nan - accuracy: 0.8858 - mse: nan
Epoch 80/150
58/58 [==============================] - 0s 523us/step - loss: nan - accuracy: 0.8858 - mse: nan
Epoch 81/150
58/58 [==============================] - 0s 627us/step - loss: nan - accuracy: 0.8858 - mse: nan
Epoch 82/150
58/58 [==============================] - 0s 602us/step - loss: nan - accuracy: 0.8858 - mse: nan
Epoch 83/150
58/58 [==============================] - 0s 588us/step - loss: nan - accuracy: 0.8858 - mse: nan
Epoch 84/150
58/58 [===

58/58 [==============================] - 0s 586us/step - loss: nan - accuracy: 0.8858 - mse: nan
Epoch 150/150
58/58 [==============================] - 0s 586us/step - loss: nan - accuracy: 0.8858 - mse: nan


In [45]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

9/9 [==============================] - 0s 617us/step - loss: nan - accuracy: 0.8982 - mse: nan


ValueError: too many values to unpack (expected 2)

In [46]:
batch_size = 128
epochs = 50

model.compile(loss="categorical_crossentropy", optimizer=Adam(epsilon=1e-07, learning_rate=0.001), metrics=["accuracy"])
log_dir= os.path.join('logs','fit_'+name,datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),'')

#Learning Rate Annealer
from tensorflow.keras.callbacks import ReduceLROnPlateau
lrr = ReduceLROnPlateau(monitor='val_accuracy',
                       factor=.01,
                       patience=3,
                       min_lr=1e-7,
                       verbose=1)
#Early stopping
es = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=8, verbose=1, mode="auto", baseline=None, restore_best_weights=True)

# tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history=model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1,callbacks=[es, lrr])

Epoch 1/50
5/5 [==============================] - 0s 25ms/step - loss: nan - accuracy: 0.8865 - val_loss: nan - val_accuracy: 0.8793
Epoch 2/50
5/5 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.8865 - val_loss: nan - val_accuracy: 0.8793
Epoch 3/50
5/5 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.8865 - val_loss: nan - val_accuracy: 0.8793
Epoch 4/50
1/5 [=====>........................] - ETA: 0s - loss: nan - accuracy: 0.9062
Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
5/5 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.8865 - val_loss: nan - val_accuracy: 0.8793
Epoch 5/50
5/5 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.8865 - val_loss: nan - val_accuracy: 0.8793
Epoch 6/50
5/5 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.8865 - val_loss: nan - val_accuracy: 0.8793
Epoch 7/50
1/5 [=====>........................] 

In [ ]:
input_shape[1]

In [ ]:
X_train